In [1]:
import database as db
import pandas as pd

In [2]:
dbconn = db.connect_to_db('database.ini')

In [3]:
year = 2020
buac = 180
acres = 100
phos = 40
potash = 30

In [4]:
class Budget:
    """
    Calculates profitability for rotation, management, and acres.
    """
    #Get basic field info
    def __init__(self, uuid, rotation, current_crop, buac, acres=1, year=2019):
        self.uuid = uuid
        self.rotation = rotation
        self.current_crop = current_crop
        self.buac = buac
        self.acres = acres
        self.expenses = {'fert':0, 'seed':0, 'preharv_mach':0, 'harv_mach':0, 'processing':0, 'chemicals':0, 'extra':0, 'labor':0}
        self.year = year

    def fert_cost(self, n_lbs, phos_lbs, potash_lbs, n_price, phos_price, potash_price):
        """cost of fertilizer per acre
        
        Arguments:
            n_lbs {float} -- pounds of nitrogen applied per acre
            phos_lbs {float} -- pounds of phosphorous applied per acre
            potash_lbs {float} -- pounds of potash/potassium applied per acre
            n_price {float} -- price of nitrogen per pound
            phos_price {float} -- price of phosphorous per pound
            potash_price {float} -- price of potash/potassium per pound
            acres {float} -- number of acres in field or area
        
        Returns:
            float -- total price of fertilizer per acre
        """
        n_cost = (n_lbs * n_price) * self.acres
        phos_cost = (phos_lbs * phos_price) * self.acres
        potash_cost = (potash_lbs * potash_price) * self.acres
        total_fert_cost = n_cost + phos_cost + potash_cost
        self.expenses['fert']=total_fert_cost
        return self.expenses

    def seed_cost(self, seed_rate, seed_price):
        """Calculates cost of seed for one acre
        
        Arguments:
            seed_rate {int} -- number of seeds planted per acre
            seed_price {float} -- cost per 1000 seeds
            acres {float} -- number of acres in field or area
        
        Returns:
            float -- cost of seed per acre
        """
        seed_cost_acre = ((seed_rate * seed_price)/1000) * self.acres
        self.expenses['seed']=seed_cost_acre
        return self.expenses

    def preharvest_machinery_cost(self, fix_chisel, var_chisel, fix_disk, var_disk, fix_applicator, var_applicator, fix_cultivator, var_cultivator, fix_planter, var_planter, fix_sprayer, var_sprayer):
        """Calculates per acre costs for preharvest machinery
        
        Arguments:
            var_chisel {float} -- variable chisel cost
            fix_chisel {float} -- fixed chisel cost
            var_disk {float} -- variable disk cost
            fix_disk {float} -- fixed disk cost
            var_applicator {float} -- variable fertilizer application cost
            fix_applicator {float} -- fixed fertilizer application cost
            var_cultivator {float} -- variable field cultivator cost
            fix_cultivator {float} -- fixed field cultivator cost
            fix_planter {float} -- fixed planter cost
            var_planter {float} -- variable planter cost
            var_sprayer {float} -- variable sprayer cost
            fix_sprayer {float} -- fixed sprayer cost
            acres {float} -- number of acres in field or area
        
        Returns:
            float -- cost of preharvest machinery per acre
        """
        tillage_cost = (var_chisel + fix_chisel + var_disk + fix_disk) * self.acres
        applicator_cost = (var_applicator + fix_applicator) * self.acres
        cultivator_cost = (var_cultivator + fix_cultivator) * self.acres
        planter_cost = (var_planter + fix_planter) * self.acres
        sprayer_cost = (var_sprayer + fix_planter) * self.acres
        total_preharvest = tillage_cost + applicator_cost + cultivator_cost + planter_cost + sprayer_cost
        self.expenses['preharv_mach']=total_preharvest
        return self.expenses

    def harvest_machinery_cost(self, fix_combine, var_combine, fix_wagon, var_wagon):
        """calculates per acre cost of harvest machinery
        
        Arguments:
            fix_combine {float} -- fixed cost of combine/harvester
            var_combine {float} -- variable cost of combine/harvester
            fix_wagon {float} -- fixed cost of wagon/grain cart
            var_wagon {float} -- variable cost of wagon/grain cart
            acres {float} -- number of acres in field or area
        
        Returns:
            float -- total cost per acre of harvest machinery
        """
        combine_cost = (fix_combine + var_combine) * self.acres
        wagon_cost = (fix_wagon + var_wagon) * self.acres
        total_harvest_machinery = combine_cost + wagon_cost
        self.expenses['harv_mach'] = total_harvest_machinery
        return self.expenses

    def processing_cost(self, fix_hauling, var_hauling, fix_drying, var_drying, fix_handling, var_handling):
        """Calculate cost per bushel of processing 
        
        Arguments:
            buac {float} -- bushels of grain per acre
            fix_hauling {float} -- fixed cost of hauling
            var_hauling {float} -- variable cost of hauling
            fix_drying {float} -- fixed cost of drying
            var_drying {float} -- variable cost drying
            fix_handling {float} -- fixed cost of handling
            var_handling {float} -- variable cost of handling
            acres {float} -- number of acres in field or area
        
        Returns:
            float -- total cost of processing per acre
        """
        hauling_cost = ((var_hauling + fix_hauling) * self.buac) * self.acres
        drying_cost = ((var_drying + fix_drying) * self.buac) * self.acres
        handling_cost = ((var_hauling + fix_hauling) * self.buac) * self.acres
        total_processing = hauling_cost + drying_cost + handling_cost
        self.expenses['processing'] = total_processing
        return self.expenses
        
    def chemicals_cost(self, herbicide, insecticide):
        """calculate cost of herbicides and pesticides per acre
        
        Arguments:
            herbicide {float} -- cost per acre of herbicide
            insecticide {float} -- cost per acre of insecticide
            acres {float} -- number of acres in field or area
        
        Returns:
            float -- cost of herbicide and pesticide per acre
        """
        herb_cost = herbicide
        insect_cost = insecticide
        total_chemical = herb_cost + insect_cost
        self.expenses['chemicals'] = total_chemical
        return self.expenses


    def extra_costs(self, crop_insurance, rent_cost, rent=False):
        """calculate cost of extra costs like rent and insurance per acre
        
        Arguments:
            rent {float} -- cost of rent per acre
            crop_insurance {float} -- cost of insurance per acre
            acres {float} -- number of acres to analyze
        
        Returns:
            float -- total extra costs per acre
        """
        if rent == True:
            extra_rent_cost = (rent_cost + crop_insurance) * self.acres
            self.expenses['extra'] = extra_rent_cost
        else:
            extra_own_cost = crop_insurance * self.acres
            self.expenses['extra'] = extra_own_cost
        return self.expenses

    def labor_cost(self, labor_hours, rate_of_pay):
        """calculates cost of labor
        
        Arguments:
            labor_hours {float} -- number of hours of hired labor
            rate_of_pay {float} -- hourly rate of pay in dollars
            acres {float} -- number of acres labor was hired for
        
        Returns:
            float -- total cost of labor per acre
        """
        total_labor = (labor_hours * rate_of_pay) * self.acres
        self.expenses['labor'] = total_labor
        return self.expenses

    def calc_revenue(self, comm_price):
        """calculates total revenue from yield and commodity price
        
        Arguments:
            comm_price {float} -- current price of given commodity
            buac {float} -- yield in bushels per acre
            acres {float} -- number of acres in given field or area
        
        Returns:
            float -- total revenue from given field or area
        """
        calc = (self.buac * self.acres) * comm_price
        self.revenue = round(calc, 2)
        return self.revenue

    #TODO DOCUMENTATION
    def sum_expenses(self):
        """
        Sum total of a list that contains all expenses.
        """
        self.expense_total = sum(self.expenses.values())
        return self.expense_total
        
    def calc_profit(self):
        """[summary]
        
        Arguments:
            revenue {[type]} -- [description]
            expenses {[type]} -- [description]
        
        Keyword Arguments:
            acres {int} -- [description] (default: {1})
        """
        self.profit = self.revenue - self.expense_total
        return self.profit

In [5]:
dbconn = db.connect_to_db('database.ini')

In [68]:
runs_query = f'SELECT * FROM public.runs_samples LIMIT 1000'
runs_df = pd.read_sql(runs_query, dbconn)

In [69]:
mgmt_query = f'SELECT * FROM public.mgmt_samples'
mgmt_df = pd.read_sql(mgmt_query, dbconn)

In [70]:
mgmt_df

,tillage.implement,tillage.date,fertilizer.formulation,tillage.depth,fertilizer.crop,fertilizer.rotation,fertilizer.application_method,fertilizer.app1_date,fertilizer.app2_date,crops.cultivar,crops.sow_crop,crops.sowing_density,crops.sowing_depth,crops.row_spacing,crops.harvest_crop,tillage.residue,fertilizer.app1_kg_n_ha,fertilizer.app2_kg_n_ha,crops.planting_date,management_sample_id
0,user_defined,10-nov,uan_n,0,maize,cont-maize,single,20-apr,20-apr,B_105,maize,8,51,381,maize,0.000000,243.06679,0.0,20-apr,1
1,user_defined,10-nov,uan_n,0,maize,cont-maize,single,20-apr,20-apr,B_105,maize,8,51,381,maize,0.000000,215.32307,0.0,20-apr,2
2,user_defined,10-nov,uan_n,0,maize,cont-maize,single,20-apr,20-apr,B_105,maize,8,51,381,maize,0.000000,224.69101,0.0,20-apr,3
3,user_defined,10-nov,uan_n,0,maize,cont-maize,single,20-apr,20-apr,B_105,maize,8,51,508,maize,0.000000,220.30394,0.0,20-apr,4
4,user_defined,10-nov,uan_n,0,maize,cont-maize,single,20-apr,20-apr,B_105,maize,8,51,508,maize,0.000000,214.10484,0.0,20-apr,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
52280,user_defined,10-nov,uan_n,203,soybean,maize-soybean,split,15-jun,10-jul,MG_3,soybean,42,38,381,soybean,0.551937,0.00000,0.0,15-jun,52281
52281,user_defined,10-nov,uan_n,203,soybean,maize-soybean,split,15-jun,10-jul,MG_3,soybean,42,38,381,soybean,0.621392,0.00000,0.0,15-jun,52282
52282,user_defined,10-nov,uan_n,203,soybean,maize-soybean,split,15-jun,10-jul,MG_3,soybean,42,38,762,soybean,0.565793,0.00000,0.0,15-jun,52283
52283,user_defined,10-nov,uan_n,203,soybean,maize-soybean,split,15-jun,10-jul,MG_3,soybean,42,38,762,soybean,0.757885,0.00000,0.0,15-jun,52284


In [72]:
runs_ids = list(runs_df['management_sample_id'])

In [78]:
get_columns_query = f'SELECT * FROM public.mgmt_samples LIMIT 0'
runs_df = pd.read_sql(get_columns_query, dbconn)

In [79]:
runs_df

,tillage.implement,tillage.date,fertilizer.formulation,tillage.depth,fertilizer.crop,fertilizer.rotation,fertilizer.application_method,fertilizer.app1_date,fertilizer.app2_date,crops.cultivar,crops.sow_crop,crops.sowing_density,crops.sowing_depth,crops.row_spacing,crops.harvest_crop,tillage.residue,fertilizer.app1_kg_n_ha,fertilizer.app2_kg_n_ha,crops.planting_date,management_sample_id


In [74]:
def get_mgmt(mgmt_df, runs_ids):
    new_df = pd.DataFrame()
    for i in runs_ids:
        new_df.append(mgmt_df.loc[mgmt_df['management_sample_id'] == i])
    return new_df

In [75]:
get_mgmt(mgmt_df_2, runs_ids)

""


In [23]:
mgmt_df['fertilizer.rotation'][(mgmt_df['fertilizer.rotation'] == 'cont-maize')] = 'cc'
mgmt_df['fertilizer.rotation'][(mgmt_df['fertilizer.rotation'] == 'maize-soybean') & (mgmt_df['crops.sow_crop'] == 'maize')] = 'cfs'
mgmt_df['fertilizer.rotation'][(mgmt_df['fertilizer.rotation'] == 'maize-soybean') & (mgmt_df['crops.sow_crop'] == 'soybean')] = 'sfc'

In [56]:
mgmt_df_2 = mgmt_df
mgmt_df_2[]

,tillage.implement,tillage.date,fertilizer.formulation,tillage.depth,fertilizer.crop,fertilizer.rotation,fertilizer.application_method,fertilizer.app1_date,fertilizer.app2_date,crops.cultivar,crops.sow_crop,crops.sowing_density,crops.sowing_depth,crops.row_spacing,crops.harvest_crop,tillage.residue,fertilizer.app1_kg_n_ha,fertilizer.app2_kg_n_ha,crops.planting_date,management_sample_id
0,user_defined,10-nov,uan_n,0,maize,cont-maize,single,20-apr,20-apr,B_105,maize,8,51,381,maize,0.000000,243.06679,0.0,20-apr,1
1,user_defined,10-nov,uan_n,0,maize,cont-maize,single,20-apr,20-apr,B_105,maize,8,51,381,maize,0.000000,215.32307,0.0,20-apr,2
2,user_defined,10-nov,uan_n,0,maize,cont-maize,single,20-apr,20-apr,B_105,maize,8,51,381,maize,0.000000,224.69101,0.0,20-apr,3
3,user_defined,10-nov,uan_n,0,maize,cont-maize,single,20-apr,20-apr,B_105,maize,8,51,508,maize,0.000000,220.30394,0.0,20-apr,4
4,user_defined,10-nov,uan_n,0,maize,cont-maize,single,20-apr,20-apr,B_105,maize,8,51,508,maize,0.000000,214.10484,0.0,20-apr,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
52280,user_defined,10-nov,uan_n,203,soybean,maize-soybean,split,15-jun,10-jul,MG_3,soybean,42,38,381,soybean,0.551937,0.00000,0.0,15-jun,52281
52281,user_defined,10-nov,uan_n,203,soybean,maize-soybean,split,15-jun,10-jul,MG_3,soybean,42,38,381,soybean,0.621392,0.00000,0.0,15-jun,52282
52282,user_defined,10-nov,uan_n,203,soybean,maize-soybean,split,15-jun,10-jul,MG_3,soybean,42,38,762,soybean,0.565793,0.00000,0.0,15-jun,52283
52283,user_defined,10-nov,uan_n,203,soybean,maize-soybean,split,15-jun,10-jul,MG_3,soybean,42,38,762,soybean,0.757885,0.00000,0.0,15-jun,52284
